# Getting Started with agent-workshop

Welcome to **agent-workshop**, an agentic PR automation framework with human-gated checkpoints and full observability.

## What is agent-workshop?

**Primary Feature: Triangle Workflow**
- Automated GitHub issue → PR generation
- Human-gated review checkpoints
- Greptile AI code review integration
- Per-project configuration via `.triangle.toml`

**Additional Features:**
- **Pre-built agents** for validation, code review, and releases
- **Two agent patterns**: Simple agents and LangGraph workflows
- **Dual provider system**: Claude Code SDK (dev) or Anthropic API (prod)
- **Full observability**: Automatic Langfuse tracing for all operations
- **Blueprint system**: Define agents in YAML, generate Python code

## Installation

```bash
# Basic installation
pip install agent-workshop

# With pre-built agents
pip install agent-workshop[agents]

# With Claude Code SDK support (development)
pip install agent-workshop[claude-agent]

# Full installation
pip install agent-workshop[agents,claude-agent,dev]
```

## Provider Configuration

agent-workshop supports two LLM providers:

### Option 1: Claude Code SDK (Development)

Flat $20/month pricing - ideal for development and testing.

```bash
export AGENT_WORKSHOP_ENV=development
export CLAUDE_SDK_ENABLED=true
```

### Option 2: Anthropic API (Production)

Pay-per-token pricing - ideal for production use.

```bash
export AGENT_WORKSHOP_ENV=production
export ANTHROPIC_API_KEY=sk-ant-...
```

The framework auto-detects which provider to use based on environment.

## Triangle Workflow (Primary Feature)

Automate GitHub issue → PR generation with human-gated review checkpoints.

### Quick Start

```bash
# Install agent-workshop
pip install agent-workshop

# In your repository, start a workflow for an issue
triangle start --issue 42 --repo owner/repo

# Check workflow status
triangle status

# After human review, approve to continue
triangle approve owner-repo-issue-42
```

### What Triangle Does

1. **Parse Issue** - Fetches GitHub issue, extracts requirements via LLM
2. **Setup Worktree** - Creates isolated git worktree for development
3. **Generate Code** - Uses LLM to implement the requirements
4. **Verify Code** - Runs tiered verification (lint, type check, tests)
5. **Create PR** - Pushes branch and creates draft PR
6. **CHECKPOINT** - Pauses for human + Greptile AI review
7. **Process Comments** - After approval, auto-fixes review feedback
8. **Merge** - Completes the workflow

See [08_triangle_quickstart.ipynb](./08_triangle_quickstart.ipynb) for a detailed tutorial.

In [5]:
# Check your current configuration
import os

print("Current Configuration:")
print("=" * 40)
print(f"AGENT_WORKSHOP_ENV: {os.getenv('AGENT_WORKSHOP_ENV', 'development')}")
print(f"CLAUDE_SDK_ENABLED: {os.getenv('CLAUDE_SDK_ENABLED', 'true')}")
print(f"ANTHROPIC_API_KEY: {'Set' if os.getenv('ANTHROPIC_API_KEY') else 'Not set'}")
print(f"LANGFUSE_ENABLED: {os.getenv('LANGFUSE_ENABLED', 'false')}")

Current Configuration:
AGENT_WORKSHOP_ENV: development
CLAUDE_SDK_ENABLED: true
ANTHROPIC_API_KEY: Not set
LANGFUSE_ENABLED: true


## Agent Types

### 1. Simple Agents (80% of use cases)

Single input to single output pattern. Best for:
- Document validation
- Code review
- Content analysis

```python
from agent_workshop import Agent, Config

class MyAgent(Agent):
    async def run(self, content: str) -> dict:
        messages = [{"role": "user", "content": content}]
        result = await self.complete(messages)
        return {"result": result}
```

### 2. LangGraph Workflows (15% of use cases)

Multi-step workflows with state management. Best for:
- Complex validation pipelines
- Multi-stage review processes
- Workflows with conditional logic

```python
from agent_workshop.workflows import LangGraphAgent
from langgraph.graph import StateGraph, END

class MyWorkflow(LangGraphAgent):
    def build_graph(self):
        workflow = StateGraph(dict)
        workflow.add_node("step1", self.step1)
        workflow.add_node("step2", self.step2)
        workflow.add_edge("step1", "step2")
        workflow.add_edge("step2", END)
        workflow.set_entry_point("step1")
        return workflow.compile()
```

## Pre-Built Agents

agent-workshop includes production-ready agents:

| Agent | Type | Use Case |
|-------|------|----------|
| **DeliverableValidator** | Simple | Validate documents, reports, research papers |
| **ValidationPipeline** | LangGraph | Multi-step validation with quick scan + detail |
| **CodeReviewer** | Simple | Review code for quality and security |
| **PRPipeline** | LangGraph | Security scan, Quality review, Summary |
| **ReleasePipeline** | Hybrid | Automated releases with git/PR operations |
| **NotebookValidator** | Simple | Validate Jupyter notebooks |
| **AgentBuilder** | Meta | Generate agents from YAML blueprints |

In [6]:
# Quick example: Using DeliverableValidator
from agent_workshop.agents.validators import DeliverableValidator
from agent_workshop import Config

config = Config()
validator = DeliverableValidator(config)

print(f"Validator created with provider: {validator.provider_name}")
print(f"Model: {validator.model_name}")

Validator created with provider: claude_sdk
Model: claude-sonnet


In [7]:
# Quick example: Using CodeReviewer
from agent_workshop.agents.software_dev import CodeReviewer, get_preset
from agent_workshop import Config

config = Config()

# Using default settings
reviewer = CodeReviewer(config)

# Or using a preset
preset = get_preset("security_focused")
security_reviewer = CodeReviewer(config, **preset)

print("CodeReviewer presets available:")
from agent_workshop.agents.software_dev import list_presets
for p in list_presets():
    print(f"  - {p['name']}: {p['description']}")

CodeReviewer presets available:
  - general: General-purpose code review for any language
  - security_focused: Security-focused review (OWASP, secrets, vulnerabilities)
  - python_specific: Python-specific review with PEP 8, type hints, idioms
  - javascript_specific: JavaScript/TypeScript review with modern patterns
  - quick_scan: Fast review for critical issues only


## Running an Agent

All agents follow the same async pattern:

```python
import asyncio
from agent_workshop import Config
from agent_workshop.agents.validators import DeliverableValidator

async def main():
    config = Config()
    validator = DeliverableValidator(config)
    
    result = await validator.run("Your content to validate...")
    print(result)

# Run the agent
asyncio.run(main())
```

Results are automatically traced to Langfuse when `LANGFUSE_ENABLED=true`.

## Tutorial Notebooks

This is part of a series of tutorials:

### Triangle Workflow (Primary)
- **[08_triangle_quickstart.ipynb](./08_triangle_quickstart.ipynb)** - Getting started with Triangle CLI
- **[09_triangle_configuration.ipynb](./09_triangle_configuration.ipynb)** - Customizing with .triangle.toml

### Pre-Built Agents
1. **[01_deliverable_validator.ipynb](./01_deliverable_validator.ipynb)** - Document validation with presets
2. **[02_validation_pipeline.ipynb](./02_validation_pipeline.ipynb)** - Multi-step validation workflow
3. **[03_code_reviewer.ipynb](./03_code_reviewer.ipynb)** - Code review agent
4. **[04_pr_pipeline.ipynb](./04_pr_pipeline.ipynb)** - PR review workflow
5. **[05_release_pipeline.ipynb](./05_release_pipeline.ipynb)** - Automated release workflow
6. **[06_notebook_validator.ipynb](./06_notebook_validator.ipynb)** - Jupyter notebook validation
7. **[07_blueprint_system.ipynb](./07_blueprint_system.ipynb)** - Creating agents from YAML

## Next Steps

1. **Try Triangle workflow**: Start with [08_triangle_quickstart.ipynb](./08_triangle_quickstart.ipynb) to automate PRs
2. **Configure Triangle**: Customize with [09_triangle_configuration.ipynb](./09_triangle_configuration.ipynb)
3. **Explore validation agents**: See [01_deliverable_validator.ipynb](./01_deliverable_validator.ipynb)
4. **Build custom agents**: Learn from [07_blueprint_system.ipynb](./07_blueprint_system.ipynb)